In [1]:
# импортируем библиотеки numpy и pandas
import numpy as np
import pandas as pd
# импортируем функцию train_test_split(), с помощью
# которой разбиваем данные на обучающие и тестовые,
# и класс GridSearchCV, позволяющий выполнить 
# поиск по сетке
from sklearn.model_selection import (train_test_split,
                                     GridSearchCV)
# импортируем класс Pipeline, позволяющий 
# создавать конвейеры
from sklearn.pipeline import Pipeline
# импортируем класс OneHotEncoder, позволяющий 
# выполнить дамми-кодирование
from sklearn.preprocessing import OneHotEncoder
# импортируем класс LogisticRegression для построения
# логистической регрессии
from sklearn.linear_model import LogisticRegression
# импортируем функцию roc_auc_score()
# для вычисления AUC-ROC
from sklearn.metrics import roc_auc_score
# импортируем собственную функцию check_vars_order
# для проверки порядка столбцов в наборах
from utils import check_vars_order
# увеличиваем количество отображаемых столбцов
pd.set_option('display.max_columns', 50)

In [2]:
# загружаем и смотрим данные
data = pd.read_csv('Data/catfeatures_challenge_II_train.csv')
data.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,Bassoon,de4c57ee2,a64bc7ddf,598080a91,0256c7a4b,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0,0
1,1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,NaN,Theremin,2bb3c3e5c,3a3a936e8,1dddb8473,52ead350c,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0,0
2,2,0.0,1.0,0.0,F,N,Red,NaN,Hamster,Canada,Bassoon,b574c9841,708248125,5ddc9a726,745b909d1,NaN,3.0,NaN,Freezing,n,P,eN,5.0,9.0,0
3,3,NaN,0.0,0.0,F,N,Red,Circle,Hamster,Finland,Theremin,673bdf1f6,23edb8da3,3a33ef960,bdaa56dd1,f9d456e57,1.0,Novice,Lava Hot,a,C,NaN,3.0,3.0,0
4,4,0.0,NaN,0.0,T,N,Red,Triangle,Hamster,Costa Rica,NaN,777d1ac2c,3a7975e46,bc9cc2a94,NaN,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0,0


In [3]:
# смотрим типы переменных и количество 
# непропущенных наблюдений
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 25 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      600000 non-null  int64  
 1   bin_0   582106 non-null  float64
 2   bin_1   581997 non-null  float64
 3   bin_2   582070 non-null  float64
 4   bin_3   581986 non-null  object 
 5   bin_4   581953 non-null  object 
 6   nom_0   581748 non-null  object 
 7   nom_1   581844 non-null  object 
 8   nom_2   581965 non-null  object 
 9   nom_3   581879 non-null  object 
 10  nom_4   581965 non-null  object 
 11  nom_5   582222 non-null  object 
 12  nom_6   581869 non-null  object 
 13  nom_7   581997 non-null  object 
 14  nom_8   582245 non-null  object 
 15  nom_9   581927 non-null  object 
 16  ord_0   581712 non-null  float64
 17  ord_1   581959 non-null  object 
 18  ord_2   581925 non-null  object 
 19  ord_3   582084 non-null  object 
 20  ord_4   582070 non-null  object 
 21  ord_5   58

In [4]:
# пишем функцию предварительной подготовки
def preprocessing(df, newdf=False):

    # избавляемся от редких категорий
    for col in ['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']:
        abs_freq = df[col].value_counts(dropna=False)
        df[col] = np.where(df[col].isin(
            abs_freq[abs_freq >= 50].index.tolist()), 
                           df[col], 'Other')
        
    # удаляем идентификатор   
    df.drop('id', axis=1, inplace=True)
    
    if newdf == True:
        # для всех столбцов создаем 
        # индикаторы пропусков
        for col in df.columns:
            df[col + '_isnan'] = np.where(
                df[col].isnull(), 'T', 'F')
    else:
        # для всех столбцов, кроме target, 
        # создаем индикаторы пропусков        
        for col in df.columns.difference(['target']):
            df[col + '_isnan'] = np.where(
                df[col].isnull(), 'T', 'F')
        
    # создаем две новые переменные на основе переменной ord5,
    # просто извлекая первый и второй символы
    df['ord_5a'] = df['ord_5'].str[0]
    df['ord_5b'] = df['ord_5'].str[1]
    
    if newdf == True:
        # присваиваем всем переменным тип str, пропуски
        # сформируют отдельную категорию
        for col in df.columns:
            df[col] = df[col].astype('str')
    else:
        # присваиваем всем переменным, кроме target, тип str,
        # пропуски сформируют отдельную категорию
        for col in df.columns.difference(['target']):
            df[col] = df[col].astype('str')
    return df

In [5]:
# применяем нашу функцию предобработки
data = preprocessing(data)
data.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target,bin_0_isnan,bin_1_isnan,bin_2_isnan,bin_3_isnan,bin_4_isnan,day_isnan,month_isnan,nom_0_isnan,nom_1_isnan,nom_2_isnan,nom_3_isnan,nom_4_isnan,nom_5_isnan,nom_6_isnan,nom_7_isnan,nom_8_isnan,nom_9_isnan,ord_0_isnan,ord_1_isnan,ord_2_isnan,ord_3_isnan,ord_4_isnan,ord_5_isnan,ord_5a,ord_5b
0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,Bassoon,de4c57ee2,a64bc7ddf,598080a91,0256c7a4b,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0,0,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,P,w
1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,nan,Theremin,2bb3c3e5c,3a3a936e8,1dddb8473,52ead350c,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0,0,F,F,F,F,F,F,F,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,p,E
2,0.0,1.0,0.0,F,N,Red,nan,Hamster,Canada,Bassoon,b574c9841,708248125,5ddc9a726,745b909d1,nan,3.0,nan,Freezing,n,P,eN,5.0,9.0,0,F,F,F,F,F,F,F,F,T,F,F,F,F,F,F,F,T,F,T,F,F,F,F,e,N
3,nan,0.0,0.0,F,N,Red,Circle,Hamster,Finland,Theremin,673bdf1f6,23edb8da3,3a33ef960,bdaa56dd1,f9d456e57,1.0,Novice,Lava Hot,a,C,nan,3.0,3.0,0,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,T,nan,nan
4,0.0,nan,0.0,T,N,Red,Triangle,Hamster,Costa Rica,nan,777d1ac2c,3a7975e46,bc9cc2a94,nan,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0,0,F,T,F,F,F,F,F,F,F,F,F,T,F,F,F,T,F,F,F,F,F,F,F,O,Z


In [6]:
# снова смотрим типы переменных и 
# количество непропущенных значений
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 49 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   bin_0        600000 non-null  object
 1   bin_1        600000 non-null  object
 2   bin_2        600000 non-null  object
 3   bin_3        600000 non-null  object
 4   bin_4        600000 non-null  object
 5   nom_0        600000 non-null  object
 6   nom_1        600000 non-null  object
 7   nom_2        600000 non-null  object
 8   nom_3        600000 non-null  object
 9   nom_4        600000 non-null  object
 10  nom_5        600000 non-null  object
 11  nom_6        600000 non-null  object
 12  nom_7        600000 non-null  object
 13  nom_8        600000 non-null  object
 14  nom_9        600000 non-null  object
 15  ord_0        600000 non-null  object
 16  ord_1        600000 non-null  object
 17  ord_2        600000 non-null  object
 18  ord_3        600000 non-null  object
 19  or

In [7]:
# разбиваем данные на обучающие и тестовые: получаем 
# обучающий массив признаков, тестовый массив признаков, 
# обучающий массив меток, тестовый массив меток
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('target', axis=1), 
    data['target'], 
    test_size=0.3,
    stratify=data['target'],
    random_state=42)

In [8]:
# создаем итоговый конвейер
ml_pipe = Pipeline([
    ('ohe', OneHotEncoder(sparse=True, 
                          handle_unknown='ignore')), 
    ('logreg', LogisticRegression(solver='liblinear'))
])

# задаем сетку гиперпараметров
param_grid = {'logreg__C': [0.01, 0.05, 0.1, 0.5, 1]}

In [9]:
# создаем экземпляр класса GridSearchCV
gs = GridSearchCV(ml_pipe, 
                  param_grid,
                  scoring='roc_auc',
                  cv=5)

# выполняем поиск по всем значениям сетки
gs.fit(X_train, y_train);

# смотрим наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров: {}".format(
    gs.best_params_))
# оцениваем наилучшее значение AUC
print("Наилучшее значение AUC: {:.3f}".format(
    roc_auc_score(y_train, gs.predict_proba(X_train)[:, 1])))
# оцениваем AUC модели на тестовой выборке
print("AUC-ROC на тестовой выборке: {:.3f}".format(
    roc_auc_score(y_test, gs.predict_proba(X_test)[:, 1])))

Наилучшие значения гиперпараметров: {'logreg__C': 0.05}
Наилучшее значение AUC: 0.798
AUC-ROC на тестовой выборке: 0.790


In [10]:
# загружаем данные
fulldata = pd.read_csv('Data/catfeatures_challenge_II_train.csv')

In [11]:
# применяем нашу функцию предобработки
fulldata = preprocessing(fulldata)
fulldata.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target,bin_0_isnan,bin_1_isnan,bin_2_isnan,bin_3_isnan,bin_4_isnan,day_isnan,month_isnan,nom_0_isnan,nom_1_isnan,nom_2_isnan,nom_3_isnan,nom_4_isnan,nom_5_isnan,nom_6_isnan,nom_7_isnan,nom_8_isnan,nom_9_isnan,ord_0_isnan,ord_1_isnan,ord_2_isnan,ord_3_isnan,ord_4_isnan,ord_5_isnan,ord_5a,ord_5b
0,0.0,0.0,0.0,F,N,Red,Trapezoid,Hamster,Russia,Bassoon,de4c57ee2,a64bc7ddf,598080a91,0256c7a4b,02e7c8990,3.0,Contributor,Hot,c,U,Pw,6.0,3.0,0,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,P,w
1,1.0,1.0,0.0,F,Y,Red,Star,Axolotl,nan,Theremin,2bb3c3e5c,3a3a936e8,1dddb8473,52ead350c,f37df64af,3.0,Grandmaster,Warm,e,X,pE,7.0,7.0,0,F,F,F,F,F,F,F,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,p,E
2,0.0,1.0,0.0,F,N,Red,nan,Hamster,Canada,Bassoon,b574c9841,708248125,5ddc9a726,745b909d1,nan,3.0,nan,Freezing,n,P,eN,5.0,9.0,0,F,F,F,F,F,F,F,F,T,F,F,F,F,F,F,F,T,F,T,F,F,F,F,e,N
3,nan,0.0,0.0,F,N,Red,Circle,Hamster,Finland,Theremin,673bdf1f6,23edb8da3,3a33ef960,bdaa56dd1,f9d456e57,1.0,Novice,Lava Hot,a,C,nan,3.0,3.0,0,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,T,nan,nan
4,0.0,nan,0.0,T,N,Red,Triangle,Hamster,Costa Rica,nan,777d1ac2c,3a7975e46,bc9cc2a94,nan,c5361037c,3.0,Grandmaster,Cold,h,C,OZ,5.0,12.0,0,F,T,F,F,F,F,F,F,F,F,F,T,F,F,F,T,F,F,F,F,F,F,F,O,Z


In [12]:
# создаем массив меток и массив признаков
y_fulldata = fulldata.pop('target')

In [13]:
# записываем оптимальные значения гиперпараметров
best_params = gs.best_params_
# присваиваем итоговому конвейеру оптимальные 
# значения гиперпараметров
ml_pipe.set_params(**best_params)
# обучаем итоговый конвейер с оптимальными значениями 
# гиперпараметров на всех исторических данных
ml_pipe.fit(fulldata, y_fulldata);

In [14]:
# загружаем и смотрим новые данные
newdata = pd.read_csv('Data/catfeatures_challenge_II_test.csv')
newdata.head()

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,600000,0.0,0.0,0.0,F,Y,Blue,Polygon,Axolotl,Finland,Piano,52f6dd16c,147d704e4,8d857a0a1,ca9ad1d4b,fced9e114,3.0,Novice,Boiling Hot,f,U,oU,3.0,9.0
1,600001,0.0,0.0,0.0,F,Y,Red,Circle,Lion,Russia,Bassoon,691ebeae8,8653dcc2e,67a8d4ebb,060a21580,7ca8775da,1.0,Novice,Cold,n,N,NaN,2.0,8.0
2,600002,0.0,0.0,0.0,F,Y,Blue,Circle,Axolotl,Russia,Theremin,81f792c16,6cdda499e,69403e18c,165e81a00,5940334c9,1.0,Expert,Warm,i,N,DN,2.0,6.0
3,600003,1.0,0.0,0.0,F,N,Red,Polygon,Axolotl,Costa Rica,Bassoon,c9134205b,acbca4827,cb681246b,77d41330d,6fbdeefc8,1.0,Expert,Hot,m,B,AG,1.0,6.0
4,600004,0.0,0.0,1.0,F,Y,Red,Circle,NaN,Finland,Theremin,f0f100f57,6f800b9af,cd9feb5c6,2218d9dfe,2a27c8fde,1.0,Contributor,Lava Hot,o,J,DT,3.0,3.0


In [15]:
# сохраним ID для посылки
ident = newdata['id']

In [16]:
# снова смотрим типы переменных и 
# количество непропущенных значений
newdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 24 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   id      400000 non-null  int64  
 1   bin_0   388099 non-null  float64
 2   bin_1   387962 non-null  float64
 3   bin_2   388028 non-null  float64
 4   bin_3   388049 non-null  object 
 5   bin_4   388049 non-null  object 
 6   nom_0   387938 non-null  object 
 7   nom_1   388053 non-null  object 
 8   nom_2   387821 non-null  object 
 9   nom_3   387824 non-null  object 
 10  nom_4   388007 non-null  object 
 11  nom_5   388088 non-null  object 
 12  nom_6   387988 non-null  object 
 13  nom_7   387997 non-null  object 
 14  nom_8   388044 non-null  object 
 15  nom_9   387940 non-null  object 
 16  ord_0   388107 non-null  float64
 17  ord_1   387833 non-null  object 
 18  ord_2   387895 non-null  object 
 19  ord_3   387947 non-null  object 
 20  ord_4   388067 non-null  object 
 21  ord_5   38

In [17]:
# применяем нашу функцию предобработки к новым данным
newdata = preprocessing(newdata, newdf=True)
newdata.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,nom_5,nom_6,nom_7,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,bin_0_isnan,bin_1_isnan,bin_2_isnan,bin_3_isnan,bin_4_isnan,nom_0_isnan,nom_1_isnan,nom_2_isnan,nom_3_isnan,nom_4_isnan,nom_5_isnan,nom_6_isnan,nom_7_isnan,nom_8_isnan,nom_9_isnan,ord_0_isnan,ord_1_isnan,ord_2_isnan,ord_3_isnan,ord_4_isnan,ord_5_isnan,day_isnan,month_isnan,ord_5a,ord_5b
0,0.0,0.0,0.0,F,Y,Blue,Polygon,Axolotl,Finland,Piano,52f6dd16c,147d704e4,8d857a0a1,ca9ad1d4b,fced9e114,3.0,Novice,Boiling Hot,f,U,oU,3.0,9.0,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,o,U
1,0.0,0.0,0.0,F,Y,Red,Circle,Lion,Russia,Bassoon,691ebeae8,8653dcc2e,67a8d4ebb,060a21580,7ca8775da,1.0,Novice,Cold,n,N,nan,2.0,8.0,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,T,F,F,nan,nan
2,0.0,0.0,0.0,F,Y,Blue,Circle,Axolotl,Russia,Theremin,81f792c16,6cdda499e,69403e18c,165e81a00,5940334c9,1.0,Expert,Warm,i,N,DN,2.0,6.0,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,D,N
3,1.0,0.0,0.0,F,N,Red,Polygon,Axolotl,Costa Rica,Bassoon,c9134205b,acbca4827,cb681246b,77d41330d,6fbdeefc8,1.0,Expert,Hot,m,B,AG,1.0,6.0,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,A,G
4,0.0,0.0,1.0,F,Y,Red,Circle,nan,Finland,Theremin,f0f100f57,6f800b9af,cd9feb5c6,2218d9dfe,2a27c8fde,1.0,Contributor,Lava Hot,o,J,DT,3.0,3.0,F,F,F,F,F,F,F,T,F,F,F,F,F,F,F,F,F,F,F,F,F,F,F,D,T


In [18]:
# снова смотрим типы переменных и 
# количество непропущенных значений
newdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 48 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   bin_0        400000 non-null  object
 1   bin_1        400000 non-null  object
 2   bin_2        400000 non-null  object
 3   bin_3        400000 non-null  object
 4   bin_4        400000 non-null  object
 5   nom_0        400000 non-null  object
 6   nom_1        400000 non-null  object
 7   nom_2        400000 non-null  object
 8   nom_3        400000 non-null  object
 9   nom_4        400000 non-null  object
 10  nom_5        400000 non-null  object
 11  nom_6        400000 non-null  object
 12  nom_7        400000 non-null  object
 13  nom_8        400000 non-null  object
 14  nom_9        400000 non-null  object
 15  ord_0        400000 non-null  object
 16  ord_1        400000 non-null  object
 17  ord_2        400000 non-null  object
 18  ord_3        400000 non-null  object
 19  or

In [19]:
# создаем списки переменных для исторического набора
# и набора новых данных
fulldata_cols = fulldata.columns.tolist()
newdata_cols = newdata.columns.tolist()

# выполняем проверку совпадения порядка столбцов
# в историческом наборе и наборе новых данных
check_vars_order(fulldata_cols, newdata_cols) 

Одни и те же списки переменных, разный порядок


In [20]:
# порядок столбцов отличается от порядка 
# столбцов в обучающей выборке, приведем
# порядок столбов в новых данных
# к порядку столбцов в исторических данных
cols_lst = fulldata.columns.tolist()
newdata = newdata[cols_lst]

In [21]:
# создаем списки переменных для исторического набора
# и набора новых данных
fulldata_cols = fulldata.columns.tolist()
newdata_cols = newdata.columns.tolist()

# выполняем проверку совпадения порядка столбцов
# в историческом наборе и наборе новых данных
check_vars_order(fulldata_cols, newdata_cols)

Одни и те же списки переменных, один и тот же порядок


In [22]:
# снова смотрим типы переменных и 
# количество непропущенных значений
newdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 48 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   bin_0        400000 non-null  object
 1   bin_1        400000 non-null  object
 2   bin_2        400000 non-null  object
 3   bin_3        400000 non-null  object
 4   bin_4        400000 non-null  object
 5   nom_0        400000 non-null  object
 6   nom_1        400000 non-null  object
 7   nom_2        400000 non-null  object
 8   nom_3        400000 non-null  object
 9   nom_4        400000 non-null  object
 10  nom_5        400000 non-null  object
 11  nom_6        400000 non-null  object
 12  nom_7        400000 non-null  object
 13  nom_8        400000 non-null  object
 14  nom_9        400000 non-null  object
 15  ord_0        400000 non-null  object
 16  ord_1        400000 non-null  object
 17  ord_2        400000 non-null  object
 18  ord_3        400000 non-null  object
 19  or

In [23]:
# вычисляем вероятности с помощью конвейера
proba = ml_pipe.predict_proba(newdata)[:, 1]

# формируем посылку для Kaggle
subm = pd.DataFrame(proba, index=ident, columns=['target'])
subm.to_csv('subm.csv')